# Sample code scripts for LangChain and LangGraph

github source:
- **[LangChain](https://github.com/langchain-ai/langchain)**
- **[LangGraph](https://github.com/langchain-ai/langgraph)**

LangChain intro: https://docs.langchain.com.cn/docs/introduction/

LangGraph intro and tutorial: https://langgraphcn.org/

## A sample test for LangGraph

reference: https://blog.csdn.net/forevercui/article/details/145876616

这是一个简单的使用 LangGraph 构建 Agent 组件的案例，用于回答用户的问题。它的结构是 
```bash
START -> chatbot -> END
```

该案例中就包含了 LangGraph 的几个基础组件
1. 状态, state: 可以是一个简单的字典或者Pydantic模型。状态包含了应用运行时需要的所有信息以及模型生成的信息;
2. 节点, node: 节点通常是Python函数,用于处理状态并返回更新后的状态;
3. 边, edge: 边定义了节点之间的连接关系和路由逻辑.

In [3]:
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import MessagesState
 
# llm = ChatOpenAI(
#     base_url="https://lxxxxx.enovo.com/v1/", 
#     api_key="sxxxxxxxwW",
#     model_name="qwen2.5-instruct"
#     )

llm = ChatOpenAI(
    base_url="https://api.deepseek.com", 
    api_key="sk-3b458ee0624f41e1b8c589e74be23e44",
    model_name="deepseek-chat"
    )

# 创建节点函数
def chatbot(state: MessagesState):
    return {"messages": [llm.invoke(state["messages"])]}

def build_graph():
    # 创建graph
    graph_builder = StateGraph(MessagesState)
    # 添加起始边
    graph_builder.add_edge(START, "chatbot")
    # 添加节点，命名为chatbot
    graph_builder.add_node("chatbot", chatbot)
    # 添加结束边
    graph_builder.add_edge("chatbot", END)
    # 代码编译为图
    graph = graph_builder.compile()
    return graph


graph = build_graph()
# 生成图的流程图片，写到当前目录下
qa_async_png = graph.get_graph().draw_mermaid_png()
with open("chatbot.png", "wb") as f:
    f.write(qa_async_png)
 
# 流式调用大模型
for event in graph.stream({"messages": [{"role": "user", "content": "中国最好的大学是哪所？"}]}):
    for value in event.values():
        print("Assistant:", value["messages"][-1].content)

Assistant: 中国有多所顶尖大学在不同领域各具优势，以下是几所公认的顶尖高校及其特点：

1. **清华大学**  
   - **优势学科**：工程、计算机科学、材料科学、建筑、经济管理等。  
   - **国际排名**：常年位居中国内地高校榜首（如QS、THE等排名）。  
   - **特色**：以理工科见长，被誉为“中国MIT”，科研实力和产学研结合突出。

2. **北京大学**  
   - **优势学科**：人文社科、基础科学（数学、物理、化学）、医学、法学等。  
   - **国际排名**：与清华并列国内前两名，人文社科领域尤为顶尖。  
   - **特色**：综合性强，思想自由、学术底蕴深厚。

3. **其他顶尖高校**（部分领域领先）：  
   - **复旦大学**/**上海交通大学**：华东地区双雄，复旦偏文理，上交强于工科和医学。  
   - **浙江大学**：学科全面，农业工程、计算机、医学等突出。  
   - **中国科学技术大学**：理科（尤其是物理、量子科学）和前沿科技研究国际知名。  
   - **南京大学**：基础学科（天文、地质等）和人文社科传统强校。  

**选择建议**：  
- 若倾向理工科（如人工智能、航天等），清华或中科大可能是首选。  
- 若关注人文社科或医学，北大、复旦更优。  
- 地域因素：北京、上海的高校资源更集中，实习机会更多。  

**注意**：中国教育部“双一流”建设名单（如清北复交等）可作为参考，但“最好”需结合专业、个人发展目标等综合判断。


我们也可以使用 LangGraph 构建一个 RAG-Agent, RAG(Retrieval-Augmented Generation) 是一种结合了检索和生成两种能力的模型，可以用于处理复杂的问题。它的结构如图

<center><img src="../assets/langgraph/rag-agent.png"><RAG-Agent Graph></center>

source: https://langgraphcn.org/tutorials/rag/langgraph_agentic_rag/

下面仅给出使用 LangGraph 构建RAG-Agent的图流程的代码，此外我们还需要定义 `agent`, `rewrite`, `generate`, `retrieve` 四个节点函数，以及 `RAGState` 状态类，这些内容在LangGraph的官方文档中都有详细的介绍。

In [4]:
from typing import Annotated, Sequence
from typing_extensions import TypedDict

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages


class AgentState(TypedDict):
    # The add_messages function defines how an update should be processed
    # Default is to replace. add_messages says "append"
    messages: Annotated[Sequence[BaseMessage], add_messages]

In [5]:
from langgraph.graph import END, StateGraph, START
from langgraph.prebuilt import ToolNode, tools_condition
from xxx import grade_documents, agent, rewrite, generate, retriever_tool

# Define a new graph
workflow = StateGraph(AgentState)

# Define the nodes we will cycle between
workflow.add_node("agent", agent)  # agent
retrieve = ToolNode([retriever_tool])
workflow.add_node("retrieve", retrieve)  # retrieval
workflow.add_node("rewrite", rewrite)  # Re-writing the question
workflow.add_node(
    "generate", generate
)  # Generating a response after we know the documents are relevant
# Call agent node to decide to retrieve or not
workflow.add_edge(START, "agent")

# Decide whether to retrieve
workflow.add_conditional_edges(
    "agent",
    # Assess agent decision
    tools_condition,
    {
        # Translate the condition outputs to nodes in our graph
        "tools": "retrieve",
        END: END,
    },
)

# Edges taken after the `action` node is called.
workflow.add_conditional_edges(
    "retrieve",
    # Assess agent decision
    grade_documents,
)
workflow.add_edge("generate", END)
workflow.add_edge("rewrite", "agent")

# Compile
graph = workflow.compile()

ModuleNotFoundError: No module named 'xxx'

In [17]:
from langchain_core.messages import BaseMessage, AIMessage

last_output = { 'agent': { 'messages': [ AIMessage(content='根据搜索结果，我将撰写一份高质量的推文，介绍国内外大厂在Agent领域的最新研究成果。以下是推文内容：\n\n---\n\n🌟 **国内外大厂Agent研究成果概览** 🌟\n\n在智能代理（Agent）领域，国内外科技巨头正不断推动技术的边界，以下是近期的一些重要成果：\n\n1. **百度** - 百度在Agent技术方面取得了显著进展，特别是在自然语言处理和多模态交互方面。其最新发布的Agent模型在多个基准测试中表现出色，能够更自然地与用户进行对话，并提供更精准的服务。[了解更多](https://blog.csdn.net/RPAdaren/article/details/145156296)\n\n2. **智谱AI** - 智谱AI在Agent的构建和应用方面进行了深入研究，特别是在医疗和教育领域的应用。其最新研究成果显示，Agent能够显著提高医疗诊断的准确性和教育的个性化水平。[了解更多](https://aibook.ren/archives/news-2025-agent-rearch-report)\n\n3. **阿里巴巴** - 阿里巴巴在Agent技术的应用方面走在前列，特别是在电商和物流领域。其最新发布的Agent模型能够更高效地处理复杂的订单管理和物流调度，大大提升了用户体验。[了解更多](https://zhuanlan.zhihu.com/p/661790669)\n\n4. **谷歌** - 谷歌在Agent领域的研究一直走在世界前列，其最新发布的Agent模型在多任务处理和环境适应性方面取得了突破。这些模型能够在多种场景下自主学习和优化，为用户提供更加智能的服务。[了解更多](https://www.huxiu.com/article/3884949.html)\n\n5. **微软** - 微软在Agent技术的评估和优化方面进行了大量研究，其最新发布的评估工具能够更准确地衡量Agent的性能和用户体验。这些工具为开发者提供了宝贵的反馈，帮助他们不断改进Agent模型。[了解更多](https://www.infoq.cn/minibook/bTgj82D3gFJK9ZLRM5Ci)\n\n这些研究成果不仅展示了Agent技术的最新进展，也为未来的发展指明了方向。随着技术的不断成熟，Agent将在更多领域发挥重要作用，为用户带来更加智能和便捷的体验。\n\n#AI #Agent #科技创新 #智能代理\n\n---\n\n希望这份推文能够帮助你更好地了解国内外大厂在Agent领域的最新研究成果。如果有任何进一步的问题或需要更多详细信息，请随时告诉我！', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 531, 'prompt_tokens': 618, 'total_tokens': 1149, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'Qwen/Qwen2.5-72B-Instruct', 'system_fingerprint': '', 'id': '0196f6db602e49b57593689e2c1d64cb', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--7c6cc76a-482f-467f-ab5d-9f54079bbd56-0', usage_metadata={'input_tokens': 618, 'output_tokens': 531, 'total_tokens': 1149, 'input_token_details': {}, 'output_token_details': {}})]}}


In [29]:
import pprint

# 打印最终所有输出内容
if last_output:
    last_content = last_output['agent']["messages"][0].content
    print("\n===== 总输出内容（最后一次迭代的 output） =====\n")
    pprint.pprint(last_content, indent=2, width=80, depth=None, compact=False)
else:
    print("\n===== 没有输出 =====\n")


===== 总输出内容（最后一次迭代的 output） =====

('根据搜索结果，我将撰写一份高质量的推文，介绍国内外大厂在Agent领域的最新研究成果。以下是推文内容：\n'
 '\n'
 '---\n'
 '\n'
 '🌟 **国内外大厂Agent研究成果概览** 🌟\n'
 '\n'
 '在智能代理（Agent）领域，国内外科技巨头正不断推动技术的边界，以下是近期的一些重要成果：\n'
 '\n'
 '1. **百度** - '
 '百度在Agent技术方面取得了显著进展，特别是在自然语言处理和多模态交互方面。其最新发布的Agent模型在多个基准测试中表现出色，能够更自然地与用户进行对话，并提供更精准的服务。[了解更多](https://blog.csdn.net/RPAdaren/article/details/145156296)\n'
 '\n'
 '2. **智谱AI** - '
 '智谱AI在Agent的构建和应用方面进行了深入研究，特别是在医疗和教育领域的应用。其最新研究成果显示，Agent能够显著提高医疗诊断的准确性和教育的个性化水平。[了解更多](https://aibook.ren/archives/news-2025-agent-rearch-report)\n'
 '\n'
 '3. **阿里巴巴** - '
 '阿里巴巴在Agent技术的应用方面走在前列，特别是在电商和物流领域。其最新发布的Agent模型能够更高效地处理复杂的订单管理和物流调度，大大提升了用户体验。[了解更多](https://zhuanlan.zhihu.com/p/661790669)\n'
 '\n'
 '4. **谷歌** - '
 '谷歌在Agent领域的研究一直走在世界前列，其最新发布的Agent模型在多任务处理和环境适应性方面取得了突破。这些模型能够在多种场景下自主学习和优化，为用户提供更加智能的服务。[了解更多](https://www.huxiu.com/article/3884949.html)\n'
 '\n'
 '5. **微软** - '
 '微软在Agent技术的评估和优化方面进行了大量研究，其最新发布的评估工具能够更准确地衡量Agent的性能和用户体验。这些工具为开发者提供了宝贵的反馈，帮助他们不断改进Agent模型。[了解更多](